In [1]:
import os,sys, json
import magic, hashlib, os, traceback
import ntpath
from capstone import *
from capstone.x86 import *

import collections
import traceback
from elftools.elf.elffile import ELFFile

In [2]:
bin_path = '/home/raisul/DATA/x86_O2_d4/'

bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) ][0:10]

ground_truth_path = '/home/raisul/ANALYSED_DATA/objdump_x86_O2_d4/'

In [3]:
len(bin_files)

10

In [4]:


def get_ground_truth(exe_path):
    elf_file_name = os.path.basename(exe_path)
    objdump_file_path = os.path.join(ground_truth_path, elf_file_name) + '.json'
    
    with open(objdump_file_path, "r") as file:
        objdump_data = json.load(file)

    ground_truth_offsets = list(objdump_data["instructions"].keys())
    ground_truth_offsets.remove('meta')
    ground_truth_offsets = [int(i) for i in ground_truth_offsets]

    return ground_truth_offsets

In [10]:


def linear_sweep(offset_inst , target_offset):
    inst_sequence = []
    max_seq_length = 15

    
    current_offset = target_offset
    for q in range(max_seq_length):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:

                inst_sequence.insert(0, None) 
                inst_sequence.append(None)
                return  inst_sequence
                
            current_offset = current_offset + current_instruction.size
            # inst_sequence.append( current_instruction.mnemonic +' '+ current_instruction.op_str )
            inst_sequence.append( current_instruction)
            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                return inst_sequence

    return inst_sequence


for bin_file_path in bin_files:

    ground_truth_offsets = get_ground_truth(bin_file_path)
    print(ground_truth_offsets)
    
    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    print("\n"*5)
    
    with open(bin_file_path, 'rb') as f:
        elffile = ELFFile(f)
        textSection = elffile.get_section_by_name('.text').data()
        text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
        for byte_index in range(len(textSection)):
            
        
            try:    

                instruction = next(md.disasm(textSection[byte_index: ], text_section_offset + byte_index ), None)
                
                # if instruction:
                #     print("0x%x:\t%s\t%s _\t%x" %(instruction.address, instruction.mnemonic, instruction.op_str, instruction.size))
                #     print(instruction.operands[0].type , instruction.operands[0].imm)
                #     # print( dir(instruction.operands[0] ))
          
                # else:
                #     print("0x%x:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
                    
                offset_inst[text_section_offset+byte_index] = instruction

            except Exception as e:
                print(traceback.print_exc() )
                print(e)

        
        
        offset_inst = collections.OrderedDict(sorted(offset_inst.items()))

        # print(offset_inst)
        for byte_index in range(len(textSection)):
            inst_seq = linear_sweep(offset_inst ,text_section_offset + byte_index )
            print('\n\n\n','# # '*10)
            
            for inst in inst_seq:
                if inst:
                    print(inst.address, ' : ' , inst.mnemonic, inst.op_str)
                else:
                    print('Invalid')

[4096, 4100, 4104, 4111, 4114, 4116, 4118, 4122, 4128, 4134, 4140, 4144, 4148, 4153, 4158, 4160, 4164, 4169, 4174, 4176, 4180, 4186, 4192, 4196, 4202, 4208, 4212, 4218, 4224, 4228, 4232, 4236, 4238, 4243, 4250, 4255, 4259, 4262, 4265, 4268, 4271, 4274, 4277, 4280, 4282, 4285, 4290, 4292, 4296, 4297, 4304, 4308, 4310, 4313, 4314, 4317, 4321, 4322, 4323, 4326, 4328, 4335, 4341, 4342, 4352, 4359, 4366, 4369, 4371, 4378, 4381, 4383, 4385, 4392, 4393, 4400, 4407, 4414, 4417, 4420, 4424, 4428, 4431, 4434, 4436, 4443, 4446, 4448, 4450, 4456, 4457, 4464, 4468, 4475, 4477, 4478, 4486, 4489, 4491, 4498, 4503, 4508, 4515, 4516, 4517, 4520, 4521, 4528, 4532, 4540, 4544, 4548, 4552]









 # # # # # # # # # # # # # # # # # # # # 
4224  :  endbr64 
4228  :  sub rsp, 8
4232  :  mov rdi, qword ptr [rsi + 8]
4236  :  xor eax, eax
4238  :  call 0x1070
4243  :  lea rsi, [rip + 0xf6a]
4250  :  mov edi, 2
4255  :  lea edx, [rax + rax*2 + 2]
4259  :  imul edx, eax
4262  :  sub edx, 5
4265  :  imul edx, e